<a href="https://colab.research.google.com/github/SurangaPrasad/deep_learning/blob/main/transfer_learning_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Geting the setup (import libraries and setup device agnostic code)

In [ ]:
import torch
import torchvision
from torch import nn
from torchvision import transforms

# get the torch version
torch.__version__

'2.1.0+cu121'

In [ ]:
# set the device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The selected device is {device}")

The selected device is cpu


### 2. Get data and split them to test, train and validation sets

Getting data from miniImageNet (https://drive.google.com/drive/folders/17a09kkqVivZQFggCw9I_YboJ23tcexNM)

In [ ]:
import os
import gdown
from pathlib import Path
import requests
import tarfile
import zipfile

In [ ]:
def download_and_extract(data_path,image_path, url, downlod_file_name, zip_file: bool=True):
  # Setup path to data folder

  # If the image folder doesn't exist, download it and prepare it...
  if image_path.is_dir():
      print(f"{image_path} directory exists.")
  else:
      print(f"Did not find {image_path} directory, creating one...")
      image_path.mkdir(parents=True, exist_ok=True)

      if zip_file:
        # Download zip file
        with open(data_path / downlod_file_name, "wb") as f:
            request = requests.get(url, verify=False)
            print("Downloading zip file...")
            f.write(request.content)

        # Unzip zip file
        with zipfile.ZipFile(data_path / downlod_file_name, "r") as zip_ref:
            print("Unzipping zip file...")
            zip_ref.extractall(image_path)

        # Remove .zip file
        os.remove(data_path / downlod_file_name)

      else:
        # Download the tar file
        with open(data_path / downlod_file_name, "wb") as f:
            request = requests.get(url)
            print("Downloading tar file ...")
            f.write(request.content)

        # Unzip mini tar file
        with tarfile.open(data_path / downlod_file_name, "r") as tar:
            print("Unzipping tar file...")
            tar.extractall(image_path)

        # Remove .tar file
        os.remove(data_path / downlod_file_name)

In [ ]:
data_path = Path("data/")
image_path = data_path / "mini_image_net"

url = 'https://drive.usercontent.google.com/download?id=107FTosYIeBn5QbynR46YG91nHcJ70whs&export=download&authuser=0&confirm=t&uuid=60b63e1a-f494-4f24-bdb1-b55131562a39&at=APZUnTVF0wI3FBlvrHXL3r-PlZjg:1704103333738'

download_and_extract(data_path, image_path, url,"mini_image_net.tar",False)

Did not find data/mini_image_net directory, creating one...
Unzipping tar file...


### 3. Create Datasets and DataLoaders

3.1 Creating a transform for torchvision.models (auto creation)

In [ ]:
## geting the weight of pretrained model
weights = torchvision.models.ResNet18_Weights.DEFAULT
weights

ResNet18_Weights.IMAGENET1K_V1

In [ ]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_data,
    test_data,
    val_data,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):

  # Turn images into data loaders
  train_dataloader = torch.utils.data.DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = torch.utils.data.DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )
  valid_dataloader = torch.utils.data.DataLoader(
      val_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, valid_dataloader

In [ ]:
#Splitting data in to train, test and validation
originalDataset = torchvision.datasets.ImageFolder(image_path/ "train", transform = auto_transforms)
#len(originalDataset)

random_gen = torch.Generator().manual_seed(42)

test_data, train_data, val_data = torch.utils.data.random_split(originalDataset, [0.8, 0.1, 0.1] , generator=random_gen)

#getting classess
class_names = originalDataset.classes


In [ ]:


# Create training, testing and validation DataLoaders
train_dataloader, test_dataloader,valid_dataloader = create_dataloaders(train_data,
                                                                               test_data,
                                                                               val_data,
                                                                               transform=auto_transforms, # perform same data transforms on our own data as the pretrained model
                                                                               batch_size=32) # set mini-batch size to 32

train_dataloader, test_dataloader, valid_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x79d5055ab580>,
 <torch.utils.data.dataloader.DataLoader at 0x79d504f29660>)

### 4. Getting a pretrained model

Earlier we got the weight for ***resnet18*** model. So let's import the same model from [`torchvision.models`](https://pytorch.org/vision/stable/models.html#classification).

In [ ]:
model = torchvision.models.resnet18(weights, progress = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]


In [ ]:
#install torchinfo if it is not installed yet

!pip install torchinfo


In [ ]:
import torchinfo
torchinfo.summary(model,
                  input_size = [32, 3, 224, 224],
                  verbose=0,
                  col_names=["input_size", "output_size", "num_params", "trainable"],
                  col_width=20,
                  row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 1000]           --                   True
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   9,408                True
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   128                  True
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   True
│    └─BasicBlock (0)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   True
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     36,864               True
│    │    └─BatchN

### 5. Freezing the base model and changing the output layer

In [ ]:
#Freezing parameters
for param in model.parameters():
    param.requires_grad = False

Before setting the output layer, let's download the EuroSAT(RGB) data set.

In [ ]:
data_path = Path("data/")
image_path = data_path / "eurosat"
url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"

download_and_extract(data_path,image_path, url, "eurosat.zip")

Did not find data/eurosat directory, creating one...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'madm.dfki.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Unzipping zip file...


Now we can set the output layer according to the number of classes in the EuroSat data set

In [ ]:
eurosat_dataset = torchvision.datasets.ImageFolder(image_path/ "2750")
# output_shape = len(eurosat_dataset.classes)
output_shape = len(originalDataset.classes)

In [ ]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Recreate the fc layer and seed it to the target device
model.fc = torch.nn.Linear(in_features=512,
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True).to(device)

Now let's see the summary again

In [ ]:
torchinfo.summary(model,
                  input_size = [32, 3, 224, 224],
                  verbose=0,
                  col_names=["input_size", "output_size", "num_params", "trainable"],
                  col_width=20,
                  row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 64]             --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    └─BasicBlock (0)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (36,864)             False
│    │    

### 6. Model training

Since this is a multiclass classification problem, we can use nn.CrossEntropyLoss() and we use torch.optim.Adam() as our optimizer with lr=0.001

In [ ]:
# Define loss and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# evaluation function
def eval(net, data_loader):
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
    net.eval()
    correct = 0.0
    num_images = 0.0
    for i_batch, (images, labels) in enumerate(data_loader):
        if use_cuda:
            images = images.cuda()
            labels = labels.cuda()
        outs = net(images)

        loss = loss_function(outs, labels)
#         _, preds = outs.max(1)
        preds = outs.argmax(dim=1)
        correct += preds.eq(labels).sum()
        num_images += len(labels)

    acc = correct / num_images
    return acc , loss

# training function
def train(net, train_loader, valid_loader, loss_function, optimizer):

    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()

    epoches = 10

    # Create empty results dictionary
    train_val_results = {"train_loss": [],
                         "train_acc": [],
                         "val_loss": [],
                         "val_acc":[]
                        }

    for epoch in range(epoches):
        net.train()
        correct = 0.0 # used to accumulate number of correctly recognized images
        num_images = 0.0 # used to accumulate number of images
        for i_batch, (images, labels) in enumerate(train_loader):
            if use_cuda:
                images = images.cuda()
                labels = labels.cuda()

            # forward propagation
            outs = net(images)
            # backward propagation
            loss = loss_function(outs, labels)
            loss.backward()
            # calculating the accuracy
            preds = outs.argmax(dim=1)
            correct += preds.eq(labels).sum()

            # update parameters
            optimizer.step()
            optimizer.zero_grad()
            num_images += len(labels)


        acc = correct / num_images
        acc_eval,loss_eval = eval(net, valid_loader)

        train_val_results["train_loss"].append(loss)
        train_val_results["train_acc"].append(acc)
        train_val_results["val_loss"].append(loss_eval)
        train_val_results["val_acc"].append(acc_eval)

        print('epoch: %d, lr: %f, accuracy: %f, loss: %f, valid accuracy: %f' % (epoch, optimizer.param_groups[0]['lr'], acc, loss.item(), acc_eval))

    return train_val_results

In [ ]:
# Start the timer
from timeit import default_timer as timer
start_time = timer()


train_val_results = train(model, train_dataloader, valid_dataloader, loss_function, optimizer)
acc_test,loss_test = eval(model, test_dataloader)

print('accuracy on testing data: %f' % acc_test) #  a reference output is provided below:

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

KeyboardInterrupt: 

### 7.Evaluate model by plotting loss curves

In [ ]:
import matplotlib.pyplot as plt

# Plot loss curves of a model
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "val_loss": [...],
             "val_acc": [...]}
    """


    epochs = range(len(results["train_loss"]))

    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []


    for i in epochs:
      train_loss.append(results["train_loss"][i].detach().cpu().numpy())
      test_loss.append(results["val_loss"][i].detach().cpu().numpy())
      train_accuracy.append(results["train_acc"][i].detach().cpu().numpy())
      test_accuracy.append(results["val_acc"][i].detach().cpu().numpy())

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

In [ ]:
plot_loss_curves(train_val_results)

# epochs = range(len(train_val_results["train_loss"]))
# len(train_val_results["train_loss"])

### 8. Saving the model

In [ ]:
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "resnt18_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

### 8. Choose 100 images from EuroSAT dataset

In [ ]:
import random
import numpy as np

# Get all available classes in the eurosat_dataset
all_classes = eurosat_dataset.classes

# Randomly select 5 classes
selected_classes = random.sample(all_classes, k=5)





In [ ]:
loaded_model = torchvision.models.resnet18(weights, progress = True)
saved_path = "resnt18_model.pth"
loaded_model.load_state_dict(torch.load(saved_path),map_location=torch.device('cpu'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.